In [1]:
import librosa
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## Extract Features from the Normalized Audio Files:
Here’s the feature extraction code that extracts the same features (duration, amplitude, dominant frequency, spectral centroid, zero-crossing rate) but applied to your normalized dataset:

In [2]:
# Path to the normalized audio folder
audio_folder = '../dataset_normalized'

# Function to extract audio features
def extract_features(file_path):
    y, sr = librosa.load(file_path)

    # 1. Duration
    duration = librosa.get_duration(y=y, sr=sr)

    # 2. Amplitude (maximum amplitude)
    amplitude_max = np.max(np.abs(y))

    # 3. FFT (Frequency Analysis)
    fft = np.fft.fft(y)
    magnitude = np.abs(fft)
    frequency = np.fft.fftfreq(len(magnitude), 1/sr)
    # Get the dominant frequency
    dominant_frequency = frequency[np.argmax(magnitude)]

    # 4. Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()

    # 5. Zero Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y).mean()

    # Return all features as a dictionary
    return {
        'duration': duration,
        'amplitude_max': amplitude_max,
        'dominant_frequency': dominant_frequency,
        'spectral_centroid': spectral_centroid,
        'zero_crossing_rate': zero_crossing_rate
    }

# Create a list to store the audio information
audio_features = []

# Iterate through all audio files in the normalized folder
for filename in os.listdir(audio_folder):
    if filename.endswith('.wav'):  # Adjust the extension if necessary
        file_path = os.path.join(audio_folder, filename)
        features = extract_features(file_path)
        features['file_name'] = filename
        audio_features.append(features)

# Convert the features list into a pandas DataFrame
df = pd.DataFrame(audio_features)

# Save the DataFrame to a CSV file (optional)
df.to_csv('normalized_audio_features.csv', index=False)

# Show the first few rows of the DataFrame
df.head()


,duration,amplitude_max,dominant_frequency,spectral_centroid,zero_crossing_rate,file_name
0,18.0,0.110535,54.777778,515.225660,0.022901,IMG_6969.wav
1,18.0,0.131378,101.833333,1744.758346,0.081379,IMG_6955.wav
2,18.0,0.107239,102.722222,1159.044980,0.051526,IMG_6941.wav
3,18.0,0.596771,49.388889,2403.120179,0.085611,IMG_6772.wav
4,18.0,0.336731,90.277778,1703.875477,0.086095,IMG_9327.wav


## Merge Features with Labels
CSV for labels like short, medium, long):

In [6]:
# Load the feature CSV from the normalized audio data
features_df = pd.read_csv('normalized_audio_features.csv')

# Load the label CSV (make sure the column names are 'file_name', 'duration_seconds', 'label')
labels_df = pd.read_csv('audio_durations_labels.csv')

# Merge the two dataframes on 'file_name'
combined_df = pd.merge(features_df, labels_df, on='file_name')

# Show the first few rows of the combined dataframe
combined_df.head()


,duration,amplitude_max,dominant_frequency,spectral_centroid,zero_crossing_rate,file_name,duration_seconds,label
0,18.0,0.110535,54.777778,515.225660,0.022901,IMG_6969.wav,3.400000,short
1,18.0,0.131378,101.833333,1744.758346,0.081379,IMG_6955.wav,13.030023,long
2,18.0,0.107239,102.722222,1159.044980,0.051526,IMG_6941.wav,8.780000,medium
3,18.0,0.596771,49.388889,2403.120179,0.085611,IMG_6772.wav,18.310023,long
4,18.0,0.336731,90.277778,1703.875477,0.086095,IMG_9327.wav,13.530023,long


## Create a Pair Plot Colored by Label

In [ ]:
# Create a pair plot for all numerical features, colored by label
sns.pairplot(combined_df, hue='label', palette='coolwarm', markers=["o", "s", "D"])

# Add a title
plt.suptitle('Pair Plot of Audio Features Colored by Label (Short, Medium, Long)', y=1.02)

# Show the plot
plt.show()
